## Purpose

To compute the ML performance of the proposed method in near real-time.

In [1]:
import os 
import sys
import numpy as np
from collections import defaultdict
import CAN_objects.aid_message
import matplotlib.pyplot as plt


actt_path = os.path.join(os.path.join(os.path.expanduser("~"), "Projects", "CAN", "actt"))
os.chdir(actt_path)
sys.path.insert(0, "src") # add src folder to path so that files from this folder can be imported

from generalFunctions import unpickle
import subprocess

import importlib
importlib.reload(CAN_objects.aid_message)
from init_cancapture_from_canlog import init_cancap
import json
import seaborn as sns
import pandas as pd

from CAN_objects.capture import MappedCapture, MatchedCapture
import math
from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram, linkage, fcluster

from pprint import pprint
from sklearn.metrics.cluster import normalized_mutual_info_score

from clusim.clustering import Clustering, remap2match
import clusim.sim as sim

import glob
from tqdm import tqdm
import itertools
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, spearmanr
from sklearn.preprocessing import normalize, scale, MinMaxScaler, StandardScaler

## Enable the Use of Functions From the Detect Repo

In [2]:
# sys.path.insert(0, "/home/cades/Projects/CAN/detect/") # add detect folder to path so that files from this folder can be imported
sys.path.insert(0, "/home/cloud/Projects/CAN/detect/") # add detect folder to path so that files from this folder can be imported
import signal_based_preprocess_functions
print(os.getcwd())

/home/cloud/Projects/CAN/actt


## Functions

In [3]:
def from_capture_to_time_series(cap, ground_truth_dbc_path):
    
    signal_multivar_ts, timepts, aid_signal_tups = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap, ground_truth_dbc_path)

    return signal_multivar_ts, timepts, aid_signal_tups


def from_captures_to_time_series(cap_1, cap_2, ground_truth_dbc_path):
        
    signal_multivar_ts_1, timepts_1, aid_signal_tups_1 = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap_1, ground_truth_dbc_path)
    signal_multivar_ts_2, timepts_2, aid_signal_tups_2 = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap_2, ground_truth_dbc_path)

    return signal_multivar_ts_1, timepts_1, aid_signal_tups_1, signal_multivar_ts_2, timepts_2, aid_signal_tups_2


def remove_constant_signals(signal_multivar_ts):
    return signal_multivar_ts[:, ~np.all(signal_multivar_ts[1:] == signal_multivar_ts[:-1], axis=0)]


def partition_time_series(signal_multivar_ts, window_length, offset):
    
    n = signal_multivar_ts.shape[0]
    i = 0
    partition = []
    
    while (i + window_length) < n:
        partition.append(signal_multivar_ts[i: i + window_length,:])
        i = i + offset
        
    if i != n:
        partition.append(signal_multivar_ts[i:n,:])
        
    return partition
    
    
def process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window_length, offset):
    
    # First dataframe
    # Convert matrix of time series into a dataframe
    df = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts[:,index] for index, tup in enumerate(aid_signal_tups)})
    # display(df)

    # Remove columns with constant values
    df = df.loc[:, (df != df.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    # df_standardized = (df-df.mean())/df.std()
    # display(df_standardized)

    # normalization
    df_standardized = (df-df.min())/(df.max()-df.min())
    
    # Partition of data frames
    n = df_standardized.shape[0]
    i = 0
    partition = []
    
    while (i + window_length) < n:
        partition.append(df_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition.append(df_standardized.iloc[i:n, :])
        
    return partition


def process_multiple_multivariate_signals(signal_multivar_ts_1, aid_signal_tups_1, signal_multivar_ts_2, aid_signal_tups_2, window_length, offset):
    
    # First dataframe
    # Convert matrix of time series into a dataframe
    df_1 = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts_1[:,index] for index, tup in enumerate(aid_signal_tups_1)})
    # display(df)
    print(df_1.shape)

    # Remove columns with constant values
    df_1 = df_1.loc[:, (df_1 != df_1.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    df_1_standardized = (df_1-df_1.mean())/df_1.std()
    # display(df_2_standardized)
    
    # Partition of data frames
    n = df_1_standardized.shape[0]
    i = 0
    partition_1 = []
    
    while (i + window_length) < n:
        partition_1.append(df_1_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition_1.append(df_1_standardized.iloc[i:n, :])
        
        
    # Second dataframe
    # Convert matrix of time series into a dataframe
    df_2 = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts_2[:,index] for index, tup in enumerate(aid_signal_tups_2)})
    # display(df)
    print(df_2.shape)

    # Remove columns with constant values
    df_2 = df_2.loc[:, (df_2 != df_2.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    df_2_standardized = (df_2-df_2.mean())/df_2.std()
    # display(df_2_standardized)
    
    # Partition of data frames
    n = df_2_standardized.shape[0]
    i = 0
    partition_2 = []
    
    while (i + window_length) < n:
        partition_2.append(df_2_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition_2.append(df_2_standardized.iloc[i:n, :])
        
    return partition_1, partition_2


def upper(df):
    '''Returns the upper triangle of a correlation matrix.
    You can use scipy.spatial.distance.squareform to recreate matrix from upper triangle.
    Args:
      df: pandas or numpy correlation matrix
    Returns:
      list of values from upper triangle
    '''
    try:
        assert(type(df) == np.ndarray)
    except:
        if type(df) == pd.DataFrame:
            df = df.values
        else:
            raise TypeError('Must be np.ndarray or pd.DataFrame')
    mask = np.triu_indices(df.shape[0], k=1)
    
    return df[mask]



def randomized_test_permutations(m1, m2):
    """Nonparametric permutation testing Monte Carlo"""
    np.random.seed(0)
    rhos = []
    n_iter = 100
    true_rho, _ = spearmanr(upper(m1), upper(m2))
    # matrix permutation, shuffle the groups
    m_ids = list(m1.columns)
    m2_v = upper(m2)
    for iter in range(n_iter):
        np.random.shuffle(m_ids) # shuffle list 
        r, _ = spearmanr(upper(m1.loc[m_ids, m_ids]), m2_v)  
        rhos.append(r)
    perm_p = ((np.sum(np.abs(true_rho) <= np.abs(rhos)))+1)/(n_iter+1) # two-tailed test

    return perm_p


def compute_correlation_matrices(partition):
    
    corr_matrices = []

    for df in partition:

        # Remove columns with constant values
        df = df.loc[:, (df != df.iloc[0]).any()] 

        # Compute correlation matrix
        corr_matrices.append(df.corr(method="pearson"))
        
    return corr_matrices


def compute_similarity_from_correlation_matrices(corr_matrices):
    
    similarities = []
    
    for i in range(len(corr_matrices)-1):

        # print("raw: ", corr_matrices[i].shape, corr_matrices[i+1].shape)

        signal_names_1 = corr_matrices[i].columns.values
        signal_names_2 = corr_matrices[i+1].columns.values
        signal_names_intersection = list(set(signal_names_1).intersection(set(signal_names_2)))

        df_1 = corr_matrices[i].loc[signal_names_intersection, signal_names_intersection] 
        df_2 = corr_matrices[i+1].loc[signal_names_intersection, signal_names_intersection]
  
        # print("pro: ", df_1.shape, df_2.shape, "\n")

        similarities.append((df_1.shape[0], spearmanr(upper(df_1), upper(df_2))[0], spearmanr(upper(df_1), upper(df_2))[1]))
        
    return similarities


def compute_similarity_from_multiple_correlation_matrices(corr_matrices_1, corr_matrices_2):
    
    similarities = []
    
    if len(corr_matrices_1) <= len(corr_matrices_2):
        corr_matrices_reference = corr_matrices_1
    else:
        corr_matrices_reference = corr_matrices_2
        
    print(len(corr_matrices_reference))
            
    for i in range(len(corr_matrices_reference)):

        # print("raw: ", corr_matrices[i].shape, corr_matrices[i+1].shape)

        signal_names_1 = corr_matrices_1[i].columns.values
        signal_names_2 = corr_matrices_2[i].columns.values
        signal_names_intersection = list(set(signal_names_1).intersection(set(signal_names_2)))

        df_1 = corr_matrices_1[i].loc[signal_names_intersection, signal_names_intersection] 
        df_2 = corr_matrices_2[i].loc[signal_names_intersection, signal_names_intersection]
  
        # print("pro: ", df_1.shape, df_2.shape, "\n")

        # similarities.append((df_1.shape[0], spearmanr(upper(df_1), upper(df_2))[0], spearmanr(upper(df_1), upper(df_2))[1]))
        
        correlation = spearmanr(upper(df_1), upper(df_2))[0]
        p_value = spearmanr(upper(df_1), upper(df_2))[1]
        
        if p_value > 0.05:
            similarities.append((i, correlation, p_value))
        else:
            similarities.append(i)
            
        
    return similarities


def create_time_intervals(total_length, window, offset):
    
    # Partition of data frames
    i = 0
    intervals = []
    
    while (i + window) < total_length:
        intervals.append((i, i + window))
        i = i + offset
        
    if i != total_length:
        intervals.append((i , total_length))
        
    return intervals


    # # Partition of data frames
    # n = df_standardized.shape[0]
    # i = 0
    # partition = []
    
    # while (i + window_length) < n:
    #     partition.append(df_standardized.iloc[i:i + window_length, :])
    #     i = i + offset
        
    # if i != n:
    #     partition.append(df_standardized.iloc[i:n, :])
        
    # return partition
    

    # intervals = []
    # # offset = 0.1*offset
    
    # for i in np.arange(0, total_length - window + 1, offset, dtype=float):
    #     intervals.append((i, i + window))

    # if i + window < total_length:
    #     intervals.append((i + offset, total_length))

    # return intervals  

## File Names

In [4]:
ground_truth_dbc_path = os.path.join(actt_path, "metadata", "dbcs", "heuristic_labeled", "anonymized_020822_030640.dbc")
#testing_captures = ["correlated_masquerade_1_030804_082640", "correlated_masquerade_2_031128_011320", "correlated_masquerade_3_040322_190000"]

# training_captures = [directory for directory in os.listdir("/home/cades/Projects/CAN/actt/data-cancaptures/") if ("road_ambient_dyno" in directory) or ("road_ambient_highway" in directory)]
training_captures = [directory for directory in os.listdir("/home/cloud/Projects/CAN/actt/data-cancaptures/") if ("road_ambient_dyno" in directory) or ("road_ambient_highway" in directory)]
print(len(training_captures), training_captures, "\n")  

testing_captures = ["correlated_masquerade_1_030804_082640", "correlated_masquerade_2_031128_011320", "correlated_masquerade_3_040322_190000", 
                    "road_attack_max_speedometer_attack_1_masquerade_060215_054000", "road_attack_max_speedometer_attack_2_masquerade_060611_002640", 
                    "road_attack_max_speedometer_attack_3_masquerade_061004_181320", "road_attack_max_engine_coolant_temp_attack_masquerade_041109_063320",
                    "road_attack_reverse_light_on_attack_1_masquerade_091205_030000", "road_attack_reverse_light_on_attack_2_masquerade_100330_214640", 
                    "road_attack_reverse_light_on_attack_3_masquerade_100724_153320", "road_attack_reverse_light_off_attack_1_masquerade_080110_162000", 
                    "road_attack_reverse_light_off_attack_2_masquerade_080505_110640", "road_attack_reverse_light_off_attack_3_masquerade_080829_045320"]

print(len(testing_captures), testing_captures) 

12 ['road_ambient_dyno_drive_basic_short_020822_030640', 'road_ambient_dyno_idle_radio_infotainment_030410_144000', 'road_ambient_dyno_drive_winter_030410_144000', 'road_ambient_highway_street_driving_diagnostics_031128_011320', 'road_ambient_dyno_drive_extended_short_021215_195320', 'road_ambient_highway_street_driving_long_050305_002000', 'road_ambient_dyno_drive_extended_long_040716_134640', 'road_ambient_dyno_drive_benign_anomaly_030804_082640', 'road_ambient_dyno_exercise_all_bits_030410_144000', 'road_ambient_dyno_reverse_040322_190000', 'road_ambient_dyno_drive_radio_infotainment_041109_063320', 'road_ambient_dyno_drive_basic_long_050305_002000'] 

13 ['correlated_masquerade_1_030804_082640', 'correlated_masquerade_2_031128_011320', 'correlated_masquerade_3_040322_190000', 'road_attack_max_speedometer_attack_1_masquerade_060215_054000', 'road_attack_max_speedometer_attack_2_masquerade_060611_002640', 'road_attack_max_speedometer_attack_3_masquerade_061004_181320', 'road_attack_m

## Obtain Metadata

In [5]:
# with open("/home/cades/Projects/CAN/actt/data/capture_metadata.json") as f:
with open("/home/cloud/Projects/CAN/actt/data/capture_metadata.json") as f:
    attack_metadata = json.load(f)
    
# pprint(testing_captures)
# pprint(attack_metadata)

attack_metadata_keys = ["correlated_signal_attack_1_masquerade", "correlated_signal_attack_2_masquerade", "correlated_signal_attack_3_masquerade", 
                        "max_speedometer_attack_1_masquerade", "max_speedometer_attack_2_masquerade", "max_speedometer_attack_3_masquerade",
                        "max_engine_coolant_temp_attack_masquerade", "reverse_light_on_attack_1_masquerade", "reverse_light_on_attack_2_masquerade",
                        "reverse_light_on_attack_3_masquerade", "reverse_light_off_attack_1_masquerade", "reverse_light_off_attack_2_masquerade",
                        "reverse_light_off_attack_3_masquerade"]

print(len(attack_metadata_keys))

13


In [6]:
print(attack_metadata_keys[0])
attack_metadata["correlated_signal_attack_1_masquerade"]

correlated_signal_attack_1_masquerade


{'description': 'start from driving; accelerate; start injecting; car rolls to stop; stop injecting; accelerate',
 'elapsed_sec': 33.101852,
 'injection_data_str': '595945450000FFFF',
 'injection_id': '0x6e0',
 'injection_interval': [9.191851, 30.050109],
 'modified': True,
 'on_dyno': True}

## Experiments on a Single Capture File (Distribution-Based)

In [7]:
signal_multivar_ts, timepts, aid_signal_tups = from_capture_to_time_series(training_captures[-1], ground_truth_dbc_path)
signal_multivar_ts.shape

(12510, 337)

In [8]:
signal_multivar_ts

array([[  148,     1, 15026, ...,     0,     0,     0],
       [  148,     1, 15023, ...,     0,     0,     0],
       [  148,     1, 15023, ...,     0,     0,     0],
       ...,
       [  133,     1, 15011, ...,     1,     0,     0],
       [  133,     1, 15014, ...,     1,     0,     0],
       [  133,     1, 15014, ...,     1,     0,     0]])

In [9]:
timepts

array([0.0000e+00, 1.0000e-01, 2.0000e-01, ..., 1.2507e+03, 1.2508e+03,
       1.2509e+03])

In [10]:
aid_signal_tups

[(14, 0, 'Unknown_0'),
 (14, 1, 'Unknown_1'),
 (14, 2, 'Unknown_2'),
 (51, 0, 'Unknown_0'),
 (51, 1, 'Unknown_1'),
 (51, 2, 'Unknown_2'),
 (51, 3, 'Unknown_3'),
 (51, 4, 'Unknown_4'),
 (51, 5, 'Unknown_5'),
 (51, 6, 'Unknown_6'),
 (60, 0, 'Unknown_0'),
 (60, 1, 'Unknown_1'),
 (61, 0, 'Unknown_0'),
 (61, 1, 'Unknown_1'),
 (61, 2, 'Unknown_2'),
 (61, 3, 'Unknown_3'),
 (61, 4, 'Unknown_4'),
 (167, 0, 'Unknown_0'),
 (167, 1, 'Unknown_1'),
 (167, 2, 'Unknown_2'),
 (167, 3, 'Unknown_3'),
 (167, 4, 'Unknown_4'),
 (167, 5, 'Unknown_5'),
 (167, 6, 'Unknown_6'),
 (167, 7, 'Unknown_7'),
 (167, 8, 'Unknown_8'),
 (186, 0, 'Unknown_0'),
 (186, 1, 'Unknown_1'),
 (186, 2, 'Unknown_2'),
 (186, 3, 'Unknown_3'),
 (186, 4, 'Unknown_4'),
 (186, 5, 'Unknown_5'),
 (186, 6, 'Unknown_6'),
 (186, 7, 'Unknown_7'),
 (192, 0, 'Unknown_0'),
 (192, 1, 'Unknown_1'),
 (192, 2, 'Unknown_2'),
 (208, 0, 'Unknown_0'),
 (208, 1, 'Unknown_1'),
 (208, 2, 'Unknown_2'),
 (208, 3, 'Unknown_3'),
 (208, 4, 'Unknown_4'),
 (208, 5,

In [11]:
timepts[-1]/60

20.848333333333336

In [12]:
np.diff(timepts)

array([0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1])

## Partition Time Series Benign

In [13]:
window = signal_multivar_ts.shape[0]
offset = window
partition_training = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window, offset)

In [14]:
print(len(partition_training))
partition_training[0]

1


,14_0,14_2,51_0,51_1,51_2,51_4,51_5,51_6,60_0,60_1,...,1694_3,1694_4,1760_0,1760_1,1760_2,1760_3,1788_3,1788_6,1788_8,1788_9
0,0.816327,0.492985,0.0,0.478431,0.0,0.133333,0.0,0.6,1.0,0.117647,...,0.0,1.000000,0.0,0.0,0.0,0.0,0.496897,0.247532,0.0,0.0
1,0.816327,0.492247,0.0,0.443137,0.0,0.733333,0.0,0.6,1.0,0.117647,...,0.0,1.000000,0.0,0.0,0.0,0.0,0.496897,0.247532,0.0,0.0
2,0.816327,0.492247,0.0,0.466667,0.0,0.333333,0.0,0.6,1.0,0.117647,...,0.0,1.000000,0.0,0.0,0.0,0.0,0.496649,0.247532,0.0,0.0
3,0.816327,0.492985,0.0,0.427451,0.0,1.000000,0.0,0.6,1.0,0.058824,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.496897,0.247532,0.0,0.0
4,0.816327,0.492247,0.0,0.450980,0.0,0.600000,0.0,0.6,1.0,0.058824,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.496897,0.247532,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12505,0.510204,0.488555,0.0,0.443137,0.0,0.733333,0.0,0.6,1.0,0.058824,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.493174,0.247532,1.0,0.0
12506,0.510204,0.489294,0.0,0.466667,0.0,0.333333,0.0,0.6,1.0,0.000000,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.493423,0.247532,1.0,0.0
12507,0.510204,0.489294,0.0,0.427451,0.0,1.000000,0.0,0.6,1.0,0.058824,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.492678,0.247532,1.0,0.0
12508,0.510204,0.490032,0.0,0.450980,0.0,0.600000,0.0,0.6,1.0,0.058824,...,0.0,0.666667,0.0,0.0,0.0,0.0,0.492678,0.247532,1.0,0.0


## Compute Correlation Matrices

In [15]:
corr_matrices_training = compute_correlation_matrices(partition_training)

In [16]:
len(corr_matrices_training)

1

In [17]:
[corr_matrices_training[i].shape for i in range(len(corr_matrices_training))]

[(211, 211)]

In [18]:
corr_sample_training = np.concatenate([upper(corr_matrices_training[i]) for i in range(len(corr_matrices_training))])
len(corr_sample_training)

22155

In [19]:
corr_sample_training

array([-0.80499476,  0.05835573, -0.15690632, ..., -0.12655689,
        0.22231895, -0.01718131])

In [20]:
# fig, ax = plt.subplots()

# ax.hist(corr_sample_training, bins=100, range=(-1.5, 1.5), density=True)

# ax.set_xlabel("Similarity")
# ax.set_ylabel("Density")

# plt.show()

## Partition Time Series Attack

In [21]:
testing_captures

['correlated_masquerade_1_030804_082640',
 'correlated_masquerade_2_031128_011320',
 'correlated_masquerade_3_040322_190000',
 'road_attack_max_speedometer_attack_1_masquerade_060215_054000',
 'road_attack_max_speedometer_attack_2_masquerade_060611_002640',
 'road_attack_max_speedometer_attack_3_masquerade_061004_181320',
 'road_attack_max_engine_coolant_temp_attack_masquerade_041109_063320',
 'road_attack_reverse_light_on_attack_1_masquerade_091205_030000',
 'road_attack_reverse_light_on_attack_2_masquerade_100330_214640',
 'road_attack_reverse_light_on_attack_3_masquerade_100724_153320',
 'road_attack_reverse_light_off_attack_1_masquerade_080110_162000',
 'road_attack_reverse_light_off_attack_2_masquerade_080505_110640',
 'road_attack_reverse_light_off_attack_3_masquerade_080829_045320']

In [22]:
signal_multivar_ts, timepts, aid_signal_tups = from_capture_to_time_series(testing_captures[0], ground_truth_dbc_path)
timepts[-1]

33.9

In [23]:
partition_testing = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, 50, 10)

In [24]:
# print(len(partition_testing))
# partition_testing[-1]

In [25]:
# partition_testing[0]

## Compute Correlation Matrices

In [26]:
corr_matrices_testing = compute_correlation_matrices(partition_testing)
len(corr_matrices_testing)

30

In [27]:
# corr_matrices_testing[0].shape

In [28]:
# for i in tqdm(range(len(corr_matrices_testing))):
    
#     corr_sample_testing = upper(corr_matrices_testing[i])
    
#     p_value = mannwhitneyu(corr_sample_training, corr_sample_testing)[1]
    
#     if p_value <= 0.05:
        
#         print(i, mannwhitneyu(corr_sample_training, corr_sample_testing))
        
# #         fig, ax = plt.subplots()

# #         ax.hist(corr_sample_training, bins=50, alpha=0.5, density=True, label="Benign")
# #         ax.set_xlabel("Similarity")
# #         ax.set_ylabel("Density")
# #         ax.hist(corr_sample_testing, bins=50, alpha=0.5, density=True, label="Attack")
# #         ax.legend(loc="upper right")

# #         plt.show() 

In [29]:
display(attack_metadata_keys[0])
attack_metadata[attack_metadata_keys[0]]

'correlated_signal_attack_1_masquerade'

{'description': 'start from driving; accelerate; start injecting; car rolls to stop; stop injecting; accelerate',
 'elapsed_sec': 33.101852,
 'injection_data_str': '595945450000FFFF',
 'injection_id': '0x6e0',
 'injection_interval': [9.191851, 30.050109],
 'modified': True,
 'on_dyno': True}

## Experiments Comparing Distributions

## Hypothesis Testing (Single Attack)

In [30]:
window = 10
offset = 1

print("Processing: ", attack_metadata_keys[0])
signal_multivar_ts, timepts, aid_signal_tups = from_capture_to_time_series(testing_captures[0], ground_truth_dbc_path)
print(timepts[-1])

partition_testing = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window, offset) # Partition time series
print("intervals: ", len(partition_testing))

# display(partition_testing[0])
# display(partition_testing[1])
# display(partition_testing[-1])

corr_matrices_testing = compute_correlation_matrices(partition_testing) # Compute Correlations

# total_length = int(np.ceil(timepts[-1])) 
total_length = timepts[-1]
print("total length (s): ", total_length) 
# print(timepts[-1], total_length)
intervals_testing = create_time_intervals(total_length, window/10, offset/10)
# print(len(intervals_testing), intervals_testing)
print("attack interval (s): ", attack_metadata[attack_metadata_keys[0]]["injection_interval"][0], attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])

tp, fp, fn, tn = 0, 0, 0, 0

for index_interval in tqdm(range(len(intervals_testing))):
    
    # print(index_interval)

    # Get correlation matrices
    corr_sample_testing = upper(corr_matrices_testing[index_interval])
     
    # Do hypothesis test
    mannwhitneyu_test = mannwhitneyu(corr_sample_training, corr_sample_testing)
    # print((index_interval, len(corr_sample_training), len(corr_sample_testing), mannwhitneyu_test[0], mannwhitneyu_test[1]))

    # print(intervals_testing[index_interval][0], intervals_testing[index_interval][1], attack_metadata[attack_metadata_keys[0]]["injection_interval"][0], attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])

    if mannwhitneyu_test[1] <= 0.05: # positive detection
        if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][0])
               or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])
                   or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])):
            tp += 1
        else:
            fp += 1
    else: # negative detection
        if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][0])
               or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])
                   or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])):
            fn += 1
        else:
            tn += 1
            
# precision
if tp + fp != 0:            
    precision = tp/(tp + fp)
else:
    precision = np.nan

# recall
if tp + fn != 0:
    recall = tp/(tp + fn)
else:
    recall = np.nan

# f1
if precision + recall != 0:
    f1 = 2*((precision*recall)/(precision + recall))

else:
    f1 = np.nan

# fpr
if fp + tn != 0:
    fpr = fp/(fp + tn)
else:
    fpr = np.nan

# fnr
if fn + tp != 0:
    fnr = fn/(fn + tp)
else:
    fnr = np.nan

# mcc
if (tp+fp == 0) or (tp+fn == 0) or (tn+fp == 0) or (tn+fn == 0):
    mcc = (tp*tn) - (fp*fn)
else:
    mcc = (tp*tn - fp*fn)/(math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))

print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
print(f"precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, fpr: {fpr:.3f}, fnr: {fnr:.3f}, mcc: {mcc:.3f}")
print(f"positive_intervals: {tp+fn:.3f}, negative_intervals: {tn+fp:.3f}\n")

Processing:  correlated_signal_attack_1_masquerade
33.9
intervals:  331
total length (s):  33.9
attack interval (s):  9.191851 30.050109


100%|██████████| 330/330 [00:01<00:00, 198.75it/s]

tp: 120, tn: 46, fp: 65, fn: 99
precision: 0.649, recall: 0.548, f1: 0.594, fpr: 0.586, fnr: 0.452, mcc: -0.036
positive_intervals: 219.000, negative_intervals: 111.000



## Hypothesis Testing (All Attacks)

In [32]:
window = 10
offset = 1
signals_training = corr_matrices_training[0].columns.values

for index_attack in range(len(attack_metadata_keys)):

    print("Processing: ", attack_metadata_keys[index_attack])
    signal_multivar_ts, timepts, aid_signal_tups = from_capture_to_time_series(testing_captures[index_attack], ground_truth_dbc_path)
    
    partition_testing = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window, offset) # Partition time series
    print("intervals: ", len(partition_testing))

    # display(partition_testing[0])
    # display(partition_testing[1])
    # display(partition_testing[-1])
    
    corr_matrices_testing = compute_correlation_matrices(partition_testing) # Compute Correlations
    
    # total_length = int(np.ceil(timepts[-1]))  
    total_length = timepts[-1]
    print("total length (s): ", total_length)
    intervals_testing = create_time_intervals(total_length, window/10, offset/10)
    # print("intervals testing: ", len(intervals_testing), intervals_testing)
    print("attack interval (s): ", attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0], attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])
    
    tp, fp, fn, tn = 0, 0, 0, 0

    for index_interval in range(len(intervals_testing)):

        # Get correlation matrices
        corr_sample_testing = upper(corr_matrices_testing[index_interval])

        # Do hypothesis test
        mannwhitneyu_test = mannwhitneyu(corr_sample_training, corr_sample_testing)
        # print((index_interval, len(corr_sample_training), len(corr_sample_testing), mannwhitneyu_test[0], mannwhitneyu_test[1]))

        if mannwhitneyu_test[1] <= 0.05: # positive detection
            if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0])
                   or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])
                       or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])):
                tp += 1
            else:
                fp += 1
        else: # negative detection
            if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0])
                   or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])
                       or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])):
                fn += 1
            else:
                tn += 1
    # precision
    if tp + fp != 0:            
        precision = tp/(tp + fp)
    else:
        precision = np.nan
        
    # recall
    if tp + fn != 0:
        recall = tp/(tp + fn)
    else:
        recall = np.nan
        
    # f1
    if precision + recall != 0:
        f1 = 2*((precision*recall)/(precision + recall))
        
    else:
        f1 = np.nan
        
    # fpr
    if fp + tn != 0:
        fpr = fp/(fp + tn)
    else:
        fpr = np.nan

    # fnr
    if fn + tp != 0:
        fnr = fn/(fn + tp)
    else:
        fnr = np.nan

    # mcc
    if (tp+fp == 0) or (tp+fn == 0) or (tn+fp == 0) or (tn+fn == 0):
        mcc = (tp*tn) - (fp*fn)
    else:
        mcc = (tp*tn - fp*fn)/(math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))

    print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
    print(f"precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, fpr: {fpr:.3f}, fnr: {fnr:.3f}, mcc: {mcc:.3f}")
    print(f"positive_intervals: {tp+fn:.3f}, negative_intervals: {tn+fp:.3f}\n")

Processing:  correlated_signal_attack_1_masquerade
intervals:  331
total length (s):  33.9
attack interval (s):  9.191851 30.050109
tp: 120, tn: 46, fp: 65, fn: 99
precision: 0.649, recall: 0.548, f1: 0.594, fpr: 0.586, fnr: 0.452, mcc: -0.036
positive_intervals: 219.000, negative_intervals: 111.000

Processing:  correlated_signal_attack_2_masquerade
intervals:  281
total length (s):  28.9
attack interval (s):  6.830477 28.225908
tp: 119, tn: 34, fp: 25, fn: 102
precision: 0.826, recall: 0.538, f1: 0.652, fpr: 0.424, fnr: 0.462, mcc: 0.094
positive_intervals: 221.000, negative_intervals: 59.000

Processing:  correlated_signal_attack_3_masquerade
intervals:  161
total length (s):  16.9
attack interval (s):  4.318482 16.95706
tp: 46, tn: 22, fp: 12, fn: 81
precision: 0.793, recall: 0.362, f1: 0.497, fpr: 0.353, fnr: 0.638, mcc: 0.008
positive_intervals: 127.000, negative_intervals: 34.000

Processing:  max_speedometer_attack_1_masquerade
intervals:  881
total length (s):  88.9
attack int